In [ ]:
!pip install Bio
!pip install SeqUtils

ERROR: Could not find a version that satisfies the requirement SubsMat (from versions: none)
ERROR: No matching distribution found for SubsMat


In [ ]:
# 1
'''
Let's remeber how to use dictionaries.
Task: return a dictionary where 
    * keys are IDs of seqs from an input fasta file (prot.fasta),
    * key's values are seqs itself. 
'''

def my_own_fasta_parser(inFile):

    sequences = {}
    seq_id = ""
    
    with open(inFile) as f:
        for line in f:
            line = line.strip()
            if line[0] == ">":
                seq_id = line[1:]
            else:
                sequences[seq_id] = line
    return sequences

In [ ]:
my_own_fasta_parser("prot.fasta")

{'seq0': 'FQTWEEFSRAAEKLYLADPMKVRVVLKYRHVDGNLCIKVTDDLVCLVYRTDQAQDVKKIEKF',
 'seq1': 'KYRTWEEFTRAAEKLYQADPMKVRVVLKYRHCDGNLCIKVTDDVVCLLYRTDQAQDVKKIEKFHSQLMRLME LKVTDNKECLKFKTDQAQEAKKMEKLNNIFFTLM',
 'seq10': 'FDSWDEFVSKSVELFRNHPDTTRYVVKYRHCEGKLVLKVTDNHECLKFKTDQAQDAKKMEK',
 'seq2': 'EEYQTWEEFARAAEKLYLTDPMKVRVVLKYRHCDGNLCMKVTDDAVCLQYKTDQAQDVKKVEKLHGK',
 'seq3': 'MYQVWEEFSRAVEKLYLTDPMKVRVVLKYRHCDGNLCIKVTDNSVCLQYKTDQAQDVK',
 'seq4': 'EEFSRAVEKLYLTDPMKVRVVLKYRHCDGNLCIKVTDNSVVSYEMRLFGVQKDNFALEHSLL',
 'seq5': 'SWEEFAKAAEVLYLEDPMKCRMCTKYRHVDHKLVVKLTDNHTVLKYVTDMAQDVKKIEKLTTLLMR',
 'seq6': 'FTNWEEFAKAAERLHSANPEKCRFVTKYNHTKGELVLKLTDDVVCLQYSTNQLQDVKKLEKLSSTLLRSI',
 'seq7': 'SWEEFVERSVQLFRGDPNATRYVMKYRHCEGKLVLKVTDDRECLKFKTDQAQDAKKMEKLNNIFF',
 'seq8': 'SWDEFVDRSVQLFRADPESTRYVMKYRHCDGKLVLKVTDNKECLKFKTDQAQEAKKMEKLNNIFFTLM',
 'seq9': 'KNWEDFEIAAENMYMANPQNCRYTMKYVHSKGHILLKMSDNVKCVQYRAENMPDLKK'}

In [ ]:
# 2
'''
We have the same fasta file (prot.fasta).
Now we want to get a list with the ids of protein seqs that have 
a relative frequency higher than a given threshold for a given residue.

And don't forget to use my_own_fasta_parser function from a previous task!
'''

def my_own_residue_abundance(input_file, residue, threshold=0.2):
    
    seq_ids = []
    sequences = my_own_fasta_parser(input_file)

    for seq_id, sequence in sequences.items():
        if sequence.count(residue)/len(sequence) > threshold:
            seq_ids.append(seq_id)

    return seq_ids

In [ ]:
my_own_residue_abundance("prot.fasta", "V", threshold=0.1)

['seq0', 'seq2', 'seq3', 'seq4', 'seq5']

In [ ]:
# 3
'''
Let's practice in Bio package using.
Task:
1. read fasta file containing several DNA seqs (nucl.fasta)
2. subset seqs that have GC content > 45 and coding protein with aromaticity > 0.01
3. write a new fasta file with those protein(!) seqs
4. return the percentage of seqs that passed your filter
Hint: Bio.SeqIO, Bio.SeqRecord, Bio.SeqUtils
'''

from Bio import SeqIO, SeqUtils
from Bio import SeqRecord
from Bio.SeqUtils.ProtParam import ProteinAnalysis

def my_own_filtering(input_file, output_file, filt_gc=45, filt_arom=0.125):
    
    c = 0
    res = []
    
    with open(input_file, "r") as content:
        
        for record in SeqIO.parse(content, "fasta"):
            c+=1
            
            # calculating GC
            calc_gc = SeqUtils.GC(record.seq)

            # calculating aomaticity
            prot = record.translate()
            x = ProteinAnalysis(str(prot.seq))
            calc_arom = x.aromaticity()
            
            # filtering
            if calc_gc >= filt_gc and calc_arom >= filt_arom:
                res.append(record)

    with open(output_file, 'w+') as f:
        SeqIO.write(res, f, 'fasta')
    
    return len(res) * 100/c

In [ ]:
my_own_filtering("nucl.fasta", "new_file.fasta", filt_arom=0.01)

/usr/local/lib/python3.7/dist-packages/Bio/Seq.py:2983: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning,


28.571428571428573

In [ ]:
# 4
"""
Continue practicing in Bio package using:
Task:
complete the following code that should be able to return 
the best alignment of two amino acid seqs (pairwise2.align.globalds)
based on BLOSUM62 matrix from Bio.SubsMat.MatrixInfo.
http://rosalind.info/glossary/blosum62/
"""

from Bio import pairwise2
from Bio.SubsMat import MatrixInfo

def balign(first_seq, second_seq):

    # Load the matrix
    matrix = MatrixInfo.blosum62

    # Generate the alignments
    alns = pairwise2.align.globalds(first_seq, second_seq, matrix, -10, -1)

    # Extract the best alignment (first one in the alns list)
    top_aln = alns[0]

    # Print the alignment, ...
    aln_A, aln_B, score, begin, end = top_aln
    print(pairwise2.format_alignment(aln_A, aln_B, score, begin, end))

In [39]:
balign('ATTCGT', 'TGT')

ATTCGT
.   ||
T---GT
  Score=-1



In [44]:
# 5
""" You have some DNA sequence: AGTACTAGAGCATTCTATGGAG.
Find out which peptides could be created from it and sort them by their length.
Use as much Biopython modules as possible.
"""

from Bio.Seq import Seq

def peptides(dna):
    res = []

    for i in range(len(dna)):
        seq = str(Seq(dna[i:]).transcribe().translate(to_stop=True))
        if len(seq) > 1:
                  res.append(seq)

    res.sort(key = len)
    return res

In [45]:
peptides('AGTACTAGAGCATTCTATGGAG')

/usr/local/lib/python3.7/dist-packages/Bio/Seq.py:2983: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning,


['LW',
 'YG',
 'ME',
 'ILW',
 'FYG',
 'SME',
 'SILW',
 'AFYG',
 'HSME',
 'RAFYG',
 'EHSME',
 'TRAFYG',
 'LEHSME',
 'STRAFYG',
 'VLEHSME']

In [46]:
# 6
""" TASK: Try to create one-line function (without (!!!) using Bio package) 
that returns reverse complementary to a given sequence. 
Hint: using dictionary & list comprehensions might be helpful.
"""

def rev_compl_one_line(seq):
    return "".join([{"A":"T", "T":"A", "G":"C", "C":"G"}[x] for x in seq[::-1]])

In [47]:
rev_compl_one_line('AGTACTAGAGCATTCTATGGAG')

'CTCCATAGAATGCTCTAGTACT'